In [10]:
import requests
import json

# url = "https://www.thesportsdb.com/api/v1/json/3/eventsseason.php?id=4334&s=2023-2024" # matches of the season 2023-2024 for ligue 1
url = "https://www.thesportsdb.com/api/v1/json/3/eventsround.php?id=4334&r=16&s=2024-2025" # For league 1 the round correspond to the number of the day in the competition, there is 38 days in a season of ligue 1 
api_key = "3"

headers = {
    "X-API-KEY": f"{api_key}",
    "Content-Type": "application/json"
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    print(json.dumps(response.json(), indent=4))
else:
    print(f"Request failed with status code: {response.status_code}")

{
    "events": [
        {
            "idEvent": "2076358",
            "idAPIfootball": "1213882",
            "strEvent": "Monaco vs Paris SG",
            "strEventAlternate": "Paris SG @ Monaco",
            "strFilename": "French Ligue 1 2024-12-18 Monaco vs Paris SG",
            "strSport": "Soccer",
            "idLeague": "4334",
            "strLeague": "French Ligue 1",
            "strLeagueBadge": "https://www.thesportsdb.com/images/media/league/badge/zcafvy1719637069.png",
            "strSeason": "2024-2025",
            "strDescriptionEN": "",
            "strHomeTeam": "Monaco",
            "strAwayTeam": "Paris SG",
            "intHomeScore": "2",
            "intRound": "16",
            "intAwayScore": "4",
            "intSpectators": null,
            "strOfficial": "",
            "strTimestamp": "2024-12-18T20:00:00",
            "dateEvent": "2024-12-18",
            "dateEventLocal": "2024-12-18",
            "strTime": "20:00:00",
            "strTimeLocal

In [16]:
import requests
import json
import pandas as pd
import time

def create_match_dataset_multiple_seasons(league_id, start_season, end_season):
    """
    Crée un dataset Pandas contenant les informations des matchs de plusieurs saisons de football pour l'Olympique Lyonnais
    en utilisant l'endpoint eventsround.php et en respectant la limite de 100 requêtes par minute.

    Args:
        league_id (int): L'identifiant de la ligue dans l'API (e.g., 4334 pour la Ligue 1).
        start_season (int): L'année de début de la première saison (e.g., 2012).
        end_season (int): L'année de début de la dernière saison (e.g., 2024).

    Returns:
        pandas.DataFrame: Un DataFrame contenant les informations des matchs de l'OL, ou None si une des requêtes a échoué.
    """

    all_data = []
    ol_team_id = "133713"  # ID de l'Olympique Lyonnais
    api_key = "3"
    headers = {
        "X-API-KEY": f"{api_key}",
        "Content-Type": "application/json"
    }
    request_count = 0
    start_time = time.time()

    for year in range(start_season, end_season + 1):
        season = f"{year}-{year + 1}"
        print(f"Récupération des données pour la saison {season}...")

        for round_number in range(1, 39):  # Boucle sur les rounds (journées) de 1 à 38
            # Gestion de la limite de requêtes par minute
            if request_count >= 100:
                elapsed_time = time.time() - start_time
                if elapsed_time < 120:
                    sleep_time = 120 - elapsed_time
                    print(f"Limite de requêtes atteinte. Attente de {sleep_time:.2f} secondes...")
                    time.sleep(sleep_time)
                request_count = 0
                start_time = time.time()

            print(f"  Récupération des données pour la journée {round_number}...")
            url = f"https://www.thesportsdb.com/api/v1/json/3/eventsround.php?id={league_id}&r={round_number}&s={season}"

            response = requests.get(url, headers=headers)
            request_count += 1

            if response.status_code == 200:
                data = response.json()
                if "events" in data and data["events"]:
                    events = data["events"]
                    for event in events:
                        # Filtrer les matchs de l'OL
                        if event["idHomeTeam"] == ol_team_id or event["idAwayTeam"] == ol_team_id:
                            all_data.append({
                                "idEvent": event["idEvent"],
                                "strEvent": event["strEvent"],
                                "strSeason": event["strSeason"],
                                "intHomeScore": event["intHomeScore"],
                                "intAwayScore": event["intAwayScore"],
                                "idHomeTeam": event["idHomeTeam"],
                                "idAwayTeam": event["idAwayTeam"],
                                "strTimestamp": event["strTimestamp"],
                                "dateEvent": event["dateEvent"],
                                "strTime": event["strTime"],
                                "strLeague": event["strLeague"],
                                "idLeague": event["idLeague"],
                                "strCountry": event["strCountry"],
                            })
                else:
                    print(f"    Aucun événement trouvé pour la ligue {league_id}, la saison {season} et la journée {round_number}.")
            else:
                print(f"    Request failed with status code: {response.status_code} pour la saison {season}, journée {round_number}")
                return None  # Arrête la fonction si une requête échoue

    return pd.DataFrame(all_data)

# Exemple d'utilisation:
league_id = 4334  # Ligue 1
start_season = 2012
end_season = 2024
df = create_match_dataset_multiple_seasons(league_id, start_season, end_season)

if df is not None:
    print(df.head())
    print(df.shape)

Récupération des données pour la saison 2012-2013...
  Récupération des données pour la journée 1...
  Récupération des données pour la journée 2...
  Récupération des données pour la journée 3...
  Récupération des données pour la journée 4...
  Récupération des données pour la journée 5...
  Récupération des données pour la journée 6...
  Récupération des données pour la journée 7...
  Récupération des données pour la journée 8...
  Récupération des données pour la journée 9...
  Récupération des données pour la journée 10...
  Récupération des données pour la journée 11...
  Récupération des données pour la journée 12...
  Récupération des données pour la journée 13...
  Récupération des données pour la journée 14...
  Récupération des données pour la journée 15...
  Récupération des données pour la journée 16...
  Récupération des données pour la journée 17...
  Récupération des données pour la journée 18...
  Récupération des données pour la journée 19...
  Récupération des donnée

In [17]:
df.head(100000)

,idEvent,strEvent,strSeason,intHomeScore,intAwayScore,idHomeTeam,idAwayTeam,strTimestamp,dateEvent,strTime,strLeague,idLeague,strCountry
0,460132,Rennes vs Lyon,2012-2013,0,1,133719,133713,None,2012-08-11,18:00:00+00:00,French Ligue 1,4334,None
1,460141,Lyon vs Troyes,2012-2013,4,1,133713,134789,None,2012-08-18,14:00:00+00:00,French Ligue 1,4334,None
2,460150,Evian Thonon Gaillard vs Lyon,2012-2013,1,1,133705,133713,None,2012-08-24,17:45:00+00:00,French Ligue 1,4334,None
3,460161,Lyon vs Valenciennes,2012-2013,3,2,133713,133706,None,2012-09-01,14:00:00+00:00,French Ligue 1,4334,None
4,460178,Lyon vs Ajaccio,2012-2013,2,0,133713,133702,None,2012-09-16,14:00:00+00:00,French Ligue 1,4334,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,2076484,St Etienne vs Lyon,2024-2025,None,None,133717,133713,2025-04-20T00:00:00,2025-04-20,00:00:00,French Ligue 1,4334,France
469,2076501,Lyon vs Rennes,2024-2025,None,None,133713,133719,2025-04-27T00:00:00,2025-04-27,00:00:00,French Ligue 1,4334,France
470,2076502,Lyon vs Lens,2024-2025,None,None,133713,133822,2025-05-04T00:00:00,2025-05-04,00:00:00,French Ligue 1,4334,France
471,2076519,Monaco vs Lyon,2024-2025,None,None,133823,133713,2025-05-11T00:00:00,2025-05-11,00:00:00,French Ligue 1,4334,Monaco


In [18]:
df.to_pickle('raw_data_ol_match_ligue_1.pkl')